In [1]:
%matplotlib inline
import numpy as np
from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, IntSlider
import matplotlib.pyplot as plt
from PIL import Image
import time
import warnings
from time import sleep
warnings.filterwarnings('ignore')

In [2]:
fingerprint = np.array(Image.open('resources/fingerprint.png').convert('1'), dtype=np.uint8)

# Morphological Operations

![](resources/dilation_erosion.png)

In this task you have to implement:
1. a method to apply the operations erosion and dilation to the fingerprint image
2. a combination of both operations by applying opening and closing to the image

As structuring element, you have to implement these which are visualized in the table below, for both step 1 and step 2. 

<table style="border: 1px solid black">
    <caption style="text-align:center">Structuring Elements</caption>
    <thead>
    <tr>
        <td style="border: 1px solid black">
            3x3 Box
        </td>
        <td style="border: 1px solid black">
            3x3 Diamond
        </td>
        <td style="border: 1px solid black">
            5x5 Diamond
        </td>
    </tr>
    </thead>
    <tbody>
    <tr>
        <td style="border: 1px solid black">
            <table>
                <tr>
                    <td>•</td>
                    <td>•</td>
                    <td>•</td>
                </tr>   
                <tr>
                    <td>•</td>
                    <td style="background:red;font-weight:bold">•</td>
                    <td>•</td>
                </tr>    
                <tr>
                    <td>•</td>
                    <td>•</td>
                    <td>•</td>
                </tr>
            </table>
        </td>
        <td style="border: 1px solid black">
            <table>
                <tr>
                    <td> </td>
                    <td>•</td>
                    <td> </td>
                </tr>   
                <tr>
                    <td>•</td>
                    <td style="background:red;font-weight:bold">•</td>
                    <td>•</td>
                </tr>    
                <tr>
                    <td> </td>
                    <td>•</td>
                    <td> </td>
                </tr>
            </table>
        </td>
        <td style="border: 1px solid black">
            <table>
                <tr>
                    <td> </td>
                    <td>•</td>
                    <td>•</td>
                    <td>•</td>
                    <td> </td>
                </tr>   
                <tr>
                    <td>•</td>
                    <td>•</td>
                    <td>•</td>
                    <td>•</td>
                    <td>•</td>
                </tr> 
                <tr>
                    <td>•</td>
                    <td>•</td>
                    <td style="background:red;font-weight:bold">•</td>
                    <td>•</td>
                    <td>•</td>
                </tr> 
                <tr>
                    <td>•</td>
                    <td>•</td>
                    <td>•</td>
                    <td>•</td>
                    <td>•</td>
                </tr> 
                <tr>
                    <td> </td>
                    <td>•</td>
                    <td>•</td>
                    <td>•</td>
                    <td> </td>
                </tr> 
            </table>
        </td>
    </tr>
    </tbody>
    </table>

**Hints**
* We are working on binary images, the data type for this is np.bool
* For this task np.all and np.any will save you a lot of loops and if conditions

## Dilation and Erosion

### Solution 

In [3]:
def convolve_se(image, se, is_dilation=True):    
    se = se.astype(np.bool)
    se_x_half = se.shape[1] // 2
    se_y_half = se.shape[0] // 2
    
    pad_image = np.pad(image,((se_y_half,se_y_half),(se_x_half,se_x_half)), mode='reflect')        
    convolved_image = np.zeros_like(pad_image, dtype=np.bool)

    for y in range(0, image.shape[0]):
        for x in range(0, image.shape[1]):                           
            patch = pad_image[y:y + se.shape[1], x: x + se.shape[0]]            
            convolved_image[y, x] = np.any(patch[se]) if is_dilation else np.all(patch[se])
                                   
    return convolved_image[se_y_half:image.shape[0] + se_y_half, se_x_half:image.shape[1] + se_x_half].astype(np.uint8)

def build_se(se_type):
    if se_type == '3x3 Diamond':
        d = np.ones((3,3))
        d[[0,2,0,2],[0,2,2,0]] = 0
        return d
    elif se_type == '3x3 Box':
        return np.ones((3,3))
    elif se_type == '5x5 Diamond':
        d = np.ones((5,5))
        d[[0,4,0,4],[0,4,4,0]] = 0
        return d

### Visulization

In [4]:
@interact(se=['3x3 Diamond', '3x3 Box', '5x5 Diamond'])
def run(se):
    dilatation = convolve_se(fingerprint, build_se(se))
    erosion = convolve_se(fingerprint, build_se(se), False)
    plt.figure(figsize=(10, 5))    
    ax1 = plt.subplot(1,3,1)
    plt.title("Source")
    plt.imshow(fingerprint, cmap='gray')
    plt.subplot(1,3,2, sharex=ax1, sharey=ax1)
    plt.title("Dilation")
    plt.imshow(dilatation, cmap='gray')
    plt.subplot(1,3,3, sharex=ax1, sharey=ax1)
    plt.title("Erosion")    
    plt.imshow(erosion, cmap='gray')
    plt.show()
    plt.savefig('resources/dilation_erosion.png')

interactive(children=(Dropdown(description='se', options=('3x3 Diamond', '3x3 Box', '5x5 Diamond'), value='3x3…

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

## Opening (erosion then delation) and Closing (delation then erosion)

To remove errors in segmentations a common procedure is the implementation of opening and closing. For this task you should impelement this methods based on the structual elements from the task above.

### Visualization

In [5]:
@interact(se=['3x3 Diamond', '3x3 Box', '5x5 Diamond'])
def run(se):
    dilatation = convolve_se(fingerprint, build_se(se))
    erosion = convolve_se(fingerprint, build_se(se), False)
    
    opening = convolve_se(erosion, build_se(se))
    closing = convolve_se(dilatation, build_se(se), False)
    
    plt.figure(figsize=(10, 5))    
    ax1 = plt.subplot(1,3,1)
    plt.title("Source")
    plt.imshow(fingerprint, cmap='gray')
    plt.subplot(1,3,2, sharex=ax1, sharey=ax1)
    plt.title("Opening")
    plt.imshow(opening, cmap='gray')
    plt.subplot(1,3,3, sharex=ax1, sharey=ax1)
    plt.title("Closing")    
    plt.imshow(closing, cmap='gray')
    plt.show()

interactive(children=(Dropdown(description='se', options=('3x3 Diamond', '3x3 Box', '5x5 Diamond'), value='3x3…